<a href="https://colab.research.google.com/github/lukasbelka/datenanalyse-beispiel/blob/main/Datenanalyse_Beipsiel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>